<a href="https://colab.research.google.com/github/EleonoraBartolomucci/Fairness/blob/master/Faceplusplus/CaboFishTaco_yelp_picture_downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#da cambiare:
# folder_id 
import requests
from lxml import html
import shutil
import os
import json

def downloadUser(user_id, counter):

    #Creare la cartella sul Drive dove andranno le foto (in data > face_data > Foto_User > (nomeRistorante)10%(strategia))
    folder_id = '1sDTp0cUpFcZ2aOoJiKGsjbz5RLCxakyi'  # folder ID = CaboFishTaco_10%Random
    filename = user_id + '.jpg'
    url = 'https://www.yelp.com/user_details?userid=' + user_id

    try:
        # Find user picture from web page
        page_content = requests.get(url)
        tree = html.fromstring(page_content.content)
        if len(tree.xpath('//*[@id="wrap"]/div[2]/div[1]/div/div[2]/div[1]/div/div/div/a/img/@src')) != 0:  # Picture found
            image_url = tree.xpath('//*[@id="wrap"]/div[2]/div[1]/div/div[2]/div[1]/div/div/div/a/img/@src')[0]

            # View all folders and file in your folder
            fileList = drive.ListFile({'q': "'" + folder_id + "' in parents and trashed=false"}).GetList()

            # Create file on Google Drive
            # param id = ID of Fairness/face_data folder in my Google Drive
            drive_file = drive.CreateFile({'title': filename, 'parents': [{'id': folder_id}]})

            # Check if file already exists in Google Drive (prevents duplicates)
            for file in fileList:
                # print('Title: %s, ID: %s' % (file['title'], file['id']))
                if file['title'] == filename:  # The file already exists, then overwrite it
                    fileID = file['id']
                    drive_file = drive.CreateFile({'id': fileID, 'title': filename, 'parents': [{'id': folder_id}]})

            response = requests.get(image_url, stream=True)

            # Create a local copy of user picture
            with open(filename, 'wb') as out_file:
                shutil.copyfileobj(response.raw, out_file)
            del response

            # Upload user picture on Google Drive
            drive_file.SetContentFile(filename)  # path of local file content
            drive_file.Upload()  # Upload the file.
            # print('title: %s, mimeType: %s' % (drive_file['title'], drive_file['mimeType']))

            # Delete local user pictures
            if os.path.exists(filename):
                os.remove(filename)
                return True

            else:
                print("The file does not exist")
                return False
    except:
        print("Si è verificato un errore alla riga " + str(counter) + " del file user.json")
        return False

In [10]:
#Da cambiare:
# users_restaurant_id, il nome del csv e quante foto prendere (10% del totale delle recensioni)
# file_list mettere l'id della cartella da dove prendere le foto

#10% foto di utenti random per ristorante
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials
from google.colab import auth
import json
import csv
import random
import pandas as pd

# Connect to Google Account
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# DOWNLOAD Restaurant.csv FROM DRIVE
# users_restaurant_id = id del file csv del ristorante (si trova in data > Scraping > Reviews_Relevance > (nomeRistorante)_relevance_desc.csv)
users_restaurant_id = '1HMiEkysal_A6edG19k5PykgIAe9NYW-a'  # FILE ID, got on google drive with condivision link
download = drive.CreateFile({'id': users_restaurant_id})
download.GetContentFile('Cabo Fish Taco_relevance_desc.csv')

# DOWNLOAD my_users.csv FROM DRIVE
users_dataset_id = '1mDvkSI919Gr6FfvrvuDW1E9KkgMKJaos'  # FILE ID, got on google drive with condivision link
download = drive.CreateFile({'id': users_dataset_id})
download.GetContentFile('my_users.csv')

#leggo i due csv
restaurant = pd.read_csv('Cabo Fish Taco_relevance_desc.csv')
dataset_users = pd.read_csv('my_users.csv')

#(si trova in data > face_data > Foto_User > (nomeRistorante)_10%Random)
file_list = drive.ListFile({'q': "'1sDTp0cUpFcZ2aOoJiKGsjbz5RLCxakyi' in parents and trashed=false"}).GetList()
listaVecchia = []
for file1 in file_list:
  id = file1['title']
  listaVecchia.append(id[0:22])

#user_list = lista id utenti che ci sono anche nel dataset
user_list = []
for riga in restaurant[restaurant.user_id.isin(dataset_users.user_id)]['user_id']:
  if riga not in listaVecchia:
    user_list.append(riga)

#random_user = lista id utenti random (10% del totale)
random_user = random.choices(user_list, k=200)
cnt = 0
foto_elaborate = 0
#Passo ogni id di random_user in downloadUser per scaricare le foto degli utenti
for user_id in random_user:
  id = '%s' % user_id
  if downloadUser(id,cnt):
    foto_elaborate = foto_elaborate + 1
  cnt = cnt + 1

print(foto_elaborate)

#for cnt, line in enumerate(fp):
 # json_data = json.loads(line)
  #user_id = json_data['user_id']
  #downloadUser(user_id, cnt)

114


In [0]:
#Prime 10% di foto di utenti per ristorante per data e per rilevanza (ordinamento di yelp)

#da cambiare:
#users_restaurant_id, nome del file e quante foto prendere (10% del totale delle recensioni)

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials
from google.colab import auth
import json
import csv
import random
import pandas as pd

# Connect to Google Account
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# DOWNLOAD Restaurant.csv FROM DRIVE
# users_restaurant_id = id del file csv del ristorante (si trova in data > Scraping > Reviews_(strategia)) > (nomeRistorante)_(strategia).csv)
users_restaurant_id = '15LwpZqBUvtbuePS-7K0ezRtfUecInjx9'  # FILE ID, got on google drive with condivision link
download = drive.CreateFile({'id': users_restaurant_id})
download.GetContentFile('Tupelo Honey_date_desc.csv')

#leggo il file csv del ristorante
restaurant = pd.read_csv('Tupelo Honey_date_desc.csv')

#user_list = lista id utenti che ci sono anche nel dataset
user_list = []
cnt =0
for riga in restaurant['user_id']:
  #10% per ogni ristorante
  if cnt < 200:
    user_list.append(riga)
    print(riga)
    cnt = cnt + 1
  else:
    break

cnt = 0
for user_id in user_list:
  id = '%s' % user_id
  downloadUser(id,cnt)
  cnt = cnt + 1

_qohdxiWQ0LO3ph_12BWaw
EIMuP58fV0S_-FFPxLxvuA
5sJlO6HA_SzC_tn1rmKy4A
FbeO2SApJ6GlOy2ZT3-88Q
sqq5uqXs5NLg_fOI7DtO9Q
5S6iNexQ44RaFIJC0rRinw
JbiUKCxNdA71uJqTX3C2nQ
kZ_L8fVkG671CkMq4IHkww
4vdFVrrYxqb4ofj8RK8xvw
7BEIyQzlskOV1OAJ0nbweQ
U3KaBKDhUumAvuAB2pEtGQ
syKoxudhp7dbwbh3xrgjVQ
GaWT2KIjIxfIBMgGIm7HGw
uvEQRrQifD__-_nVRUMHHA
rO1VFc9pbkR-LebpkchULw
GGMhbbmD_4dJPe2EIOfbZw
2OO_2F2oIbv1xUBzsMV41Q
bT_7XEioRrkG3wphb_3VMg
Bm3UZP2hJItWbHDiwGtCMA
0B9rbyXYxlDnOphH5ObTMg
UolStOZ5ap72HWEwncQ-tg
Doy_uuKN-wsrC3AeJKjoqA
nG4trCwuehsoaqYrzwjm_g
7cRqJ9Z5E5eZ0FU9aAXLiA
D-4X3DGjuSvcSJXp-vFMRw
bnBMj84G6SgrnjvNF2Q0rQ
IDwjyXR4zY1fICjuG-1idw
QUlWRKWIxjwXZoKbI1c7Kw
fEYb-rRR1SYSlUnSk0ewpA
Uek-gdx5StNjBCfHXCnu_w
6HKKi1iTmUGZpATkFU7jYQ
jwfJ1N4eOBT8JYwXaaUqbg
PMEK5fq3B_YAMfR8urwBtg
ulWSvnBzR95p3dVYn4u8lg
ctqmwDxvL_D5acPc5aVOBg
u1seTs02BI9i4T3c7E5MTA
n5VPqgBcl4tKrfjzCZJ_9g
naOwv1B7wrLp0BwZEVHgWQ
mMWmr4PZgu6ar45JpNf1tQ
sBbNN-u1povtcI0akRD4pQ
1O7wcFf0uC6Rg1DjsYXdWQ
vXiKsduwwBQLQ8EhDr2SPg
8p19CLCYzyrsfoqLCksNVA
SyP_8-Zevtd